In [1]:
import sys
import numpy as np

In [2]:
from pynq import Overlay
System_Design = Overlay("./system_v1.bit")

In [3]:
System_Design.ip_dict

{'axi_bram_ctrl_0': {'addr_range': 16384,
  'device': <pynq.pl_server.device.XlnkDevice at 0xb0288b50>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'axi_bram_ctrl_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S_AXI',
  'parameters': {'BMG_INSTANCE': 'EXTERNAL',
   'C_BRAM_ADDR_WIDTH': '10',
   'C_BRAM_INST_MODE': 'EXTERNAL',
   'C_ECC': '0',
   'C_ECC_ONOFF_RESET_VALUE': '0',
   'C_ECC_TYPE': '0',
   'C_FAMILY': 'zynq',
   'C_FAULT_INJECT': '0',
   'C_MEMORY_DEPTH': '1024',
   'C_RD_CMD_OPTIMIZATION': '0',
   'C_READ_LATENCY': '1',
   'C_SINGLE_PORT_BRAM': '1',
   'C_S_AXI_ADDR_WIDTH': '14',
   'C_S_AXI_BASEADDR': '0x40000000',
   'C_S_AXI_CTRL_ADDR_WIDTH': '32',
   'C_S_AXI_CTRL_DATA_WIDTH': '32',
   'C_S_AXI_DATA_WIDTH': '128',
   'C_S_AXI_HIGHADDR': '0x40003FFF',
   'C_S_AXI_ID_WIDTH': '1',
   'C_S_AXI_PROTOCOL': 'AXI4',
   'C_S_AXI_SUPPORTS_NARROW_BURST': '0',
   'Component_Name': 'system_v1_axi_bram_ctrl_0_0',
   'DATA_WIDTH': '128',
   'ECC_ONOFF_RESET_VALUE': '0',
   

In [4]:
CDMA_ADDR = System_Design.ip_dict['axi_cdma_0']['phys_addr']
CDMA_ADDR_range = System_Design.ip_dict['axi_cdma_0']['addr_range']
BRAM0_ADDR = System_Design.ip_dict['axi_bram_ctrl_0']['phys_addr']
BRAM0_ADDR_range = System_Design.ip_dict['axi_bram_ctrl_0']['addr_range']
BRAM1_ADDR = System_Design.ip_dict['axi_bram_ctrl_1']['phys_addr']
BRAM1_ADDR_range = System_Design.ip_dict['axi_bram_ctrl_1']['addr_range']
BRAM2_ADDR = System_Design.ip_dict['axi_bram_ctrl_2']['phys_addr']
BRAM2_ADDR_range = System_Design.ip_dict['axi_bram_ctrl_2']['addr_range']
TPU_SLVREG_ADDR = System_Design.ip_dict['tpu_v5_0']['phys_addr']
TPU_SLVREG_ADDR_range = System_Design.ip_dict['tpu_v5_0']['addr_range']
print("The Base Address of CDMA is : " + hex(CDMA_ADDR))
print("The Address Range of CDMA is : " + hex(CDMA_ADDR_range))
print("The Base Address of BRAM0 is : " + hex(BRAM0_ADDR))
print("The Address Range of BRAM0 is : " + hex(BRAM0_ADDR_range))
print("The Base Address of BRAM1 is : " + hex(BRAM1_ADDR))
print("The Address Range of BRAM1 is : " + hex(BRAM1_ADDR_range))
print("The Base Address of BRAM2 is : " + hex(BRAM2_ADDR))
print("The Address Range of BRAM2 is : " + hex(BRAM2_ADDR_range))
print("The Base Address of TPU_SLVREG is : " + hex(TPU_SLVREG_ADDR))
print("The Address Range of TPUSLVREG is : " + hex(TPU_SLVREG_ADDR_range))

The Base Address of CDMA is : 0x7e200000
The Address Range of CDMA is : 0x10000
The Base Address of BRAM0 is : 0x40000000
The Address Range of BRAM0 is : 0x4000
The Base Address of BRAM1 is : 0x42000000
The Address Range of BRAM1 is : 0x4000
The Base Address of BRAM2 is : 0x44000000
The Address Range of BRAM2 is : 0x4000
The Base Address of TPU_SLVREG is : 0x43c00000
The Address Range of TPUSLVREG is : 0x10000


In [5]:
from pynq import MMIO
CDMA_MMIO = MMIO(CDMA_ADDR, CDMA_ADDR)
BRAM0_MMIO = MMIO(BRAM0_ADDR, BRAM0_ADDR_range)
BRAM1_MMIO = MMIO(BRAM1_ADDR, BRAM1_ADDR_range)
BRAM2_MMIO = MMIO(BRAM2_ADDR, BRAM2_ADDR_range)
TPU_MMIO = MMIO(TPU_SLVREG_ADDR, TPU_SLVREG_ADDR_range)

In [6]:
def im2col(input_data, filter_h, filter_w, stride=1):
  '''
  input_data shape: (H, W, C) hieght, width, channels
  '''
  H, W, C = input_data.shape
  out_h = (H - filter_h) // stride + 1  # h of output feature map
  out_w = (W - filter_w) // stride + 1  # w of output feature map
  col = np.zeros((out_h * out_w, filter_h * filter_w * C))

  # ## Padding
  # if padding == "same":

  ## Reshape
  for y in range(out_h):
    y_min = y * stride
    y_max = y_min + filter_h
    y_start = y * out_w
    for x in range(out_w):
      x_min = x * stride
      x_max = x_min + filter_w
      col[y_start + x, :] = input_data[y_min:y_max, x_min:x_max, :].reshape(1, -1)
  #print(out_h, out_w)
  return col, (out_h, out_w)  # return the shape of ofm also

In [7]:
# japoka's version of col2im
def jcol2im(input_data, img_size):
  '''
  Param:
  "input_data" is a (filter_num, hieght, weight)
  "img_size" describe output size (H, W).
  '''
  N, H, W = img_size
  img = np.zeros([N, H, W])
  for n in range(N):
    for h in range(H):
      for w in range(W):
        index = (h * W) + w
        img[n][h][w] = input_data[index][n]
  
  return img

In [8]:
def col2im(input_data, out_h, out_w):
  '''
  input_data is the result after input matrix multiply filter matrix
  out_h is the expected h after convolution
  out_w is the expected w after convolution
  out_c is the expected c after convolution, it should be the same as # of
  channels of filters, also the width of input_data
  '''
  hw, out_c = input_data.shape
  assert(hw == out_h * out_w)  # exam if the shape is consistent
  im = input_data.reshape(out_h, out_w, out_c)

  return im

In [9]:
def conv2d(input_data, filter, bias=None, stride=1, activation=True):
  '''
  Param:
  "input_data"  is an img of shape: (H, W, C)
  "filter"      is conv kernels of the layer: (H, W, C, N)
  "bias"        is an array with len == N
  "stride"      is the moving step size of filter.
  "activation"  Assign True for adding ReLu.
  Return:
  "res"         is the img after convolution
  '''
  # Use im2col first to turn input_data into matrix 
  # then apply matrix multiplication
  filter_h, filter_w, filter_c, filter_num = filter.shape
  input_mtx, (out_h, out_w) = im2col(input_data, filter_h, filter_w, stride)
  
  # Turn filters into matrix
  filter_mtx = filter.reshape(filter_h * filter_w * filter_c, filter_num)

  # yuyu: we will replace this with our accelerated version of matrix-to-matirx
  # mutiplication
  res = np.dot(input_mtx, filter_mtx)

  if bias is not None:
    assert(len(bias) == filter_num)  # Make sure bias has correct shape
    for i in range(filter_num):
      res[:, i] = res[:, i] + bias[i]

  if activation:
    res = np.maximum(res, 0)

  res = col2im(res, out_h, out_w)  # Use yuyu's version

  return res

In [10]:
# convolution for fixed point data
def conv2d_FxPt(input_data, filter, bias=None, stride=1, activation=True, fraction_length=16):
  '''
  Param:
  "input_data"  is an img of shape: (H, W, C)
  "filter"      is a conv kernel (H, W, C, N)
  "stride"      is the moving step size of filter.
  "activation"  Assign True for adding ReLu.
  "fracttion_len" is the length of fractional part. Max size is 16 avoiding overflow.
  Return:
  "res"         is the img after convolution
  '''

  # Use im2col first to turn input_data into matrix 
  # then apply matrix multiplication
  filter_h, filter_w, filter_c, filter_num = filter.shape
  input_mtx, (out_h, out_w) = im2col(input_data, filter_h, filter_w, stride)
  input_mtx = input_mtx.astype(dtype=np.int64)
  
  # Turn filters into matrix
  filter_mtx = filter.reshape(filter_h * filter_w * filter_c, filter_num)
  filter_mtx = filter_mtx.astype(dtype=np.int64)

  # yuyu: we will replace this with our accelerated version of matrix-to-matirx
  # mutiplication

  res = np.dot(input_mtx, filter_mtx)
  res[:] = res[:] >> fraction_length
  res = res.astype(dtype=np.int32)

  if bias is not None:
    bias = bias.astype(dtype=np.int32)
    for i in range(filter_num):
      res[:, i] = res[:, i] + bias[i]

  if activation:
    res = np.maximum(res, 0)

  res = col2im(res, out_h, out_w)  # Use yuyu's version

  return res

In [11]:
def maxpooling2d(input_data, pool_size=(2, 2), stride=2):
  '''
  input_data is of shape (H, W, C)
  '''
  H, W, C = input_data.shape
  pool_h, pool_w = pool_size
  out_h = (H - pool_h) // stride + 1
  out_w = (W - pool_w) // stride + 1
  res = np.zeros([out_h, out_w, C])

  for c in range(C):
    for h in range(out_h):
      for w in range(out_w):
        h0 = h * stride
        w0 = w * stride
        h1 = h0 + pool_h
        w1 = w0 + pool_w
        res[h, w, c] = np.max(input_data[h0:h1, w0:w1, c])
  
  return res

In [12]:
def dense(input_data, weights, bias=None):
  '''
  input_data is an array with len = n
  weights    is the weights of the FC layer with shape (n, m)
  bias       is the bias of the FC layer with len = m
  '''
  _, out_n = weights.shape
  np.transpose(input_data)

  ## yuyu: we can also accelerate this part? XD
  res = np.dot(input_data, weights)

  if bias is not None:
    assert(len(bias) == out_n)
    res = res + bias
  
  return res

In [13]:
# dense for fixed point data
def dense_FxPt(input_data, weights, bias=None, fraction_length=16):
  '''
  input_data is an array with len = n
  weights    is the weights of the FC layer with shape (n, m)
  bias       is the bias of the FC layer with len = m
  fracttion_length is the length of fractional part. Max size is 16 avoiding overflow.
  '''
  input_data = input_data.astype(dtype=np.int64)
  weights = weights.astype(dtype=np.int64)
  
  _, out_n = weights.shape
  np.transpose(input_data)

  ## yuyu: we can also accelerate this part? XD
  res = np.dot(input_data, weights)
  res[:] = res[:] >> fraction_length
  res = res.astype(dtype=np.int32)


  if bias is not None:
    assert(len(bias) == out_n)
    bias = bias.astype(dtype=np.int32)
    res = res + bias
  
  return res

In [14]:
def softmax(input_data):
  return np.exp(input_data) / np.sum(np.exp(input_data))

In [15]:
def inference(input_data, layers):
  '''
  input_data is an img of shape (28, 28, 1)
  layers     is layers with pretrained weights in the model
  '''
  assert(input_data.shape == (28, 28, 1))

  filter, bias = layers[0].get_weights()  # this is keras.Conv2D layer
  res = conv2d(input_data, filter, bias, stride=1, activation=True)

  res = maxpooling2d(res, pool_size=(2, 2), stride=2)

  res = res.flatten()

  weights, bias = layers[3].get_weights()  # this is keras.Dense layer
  res = dense(res, weights, bias)

  res = softmax(res)

  return res

In [16]:
def inference_FxPt(input_data, layers, fraction_length=16):
  '''
  input_data is an img of shape (28, 28, 1)
  layers     is layers with pretrained weights in the model
  '''
  assert(input_data.shape == (28, 28, 1))

  filter, bias = layers[0]  # this is fixed-point weights
  res = conv2d_FxPt(input_data, filter, bias, stride=1, activation=True,
                    fraction_length=fraction_length)

  res = maxpooling2d(res, pool_size=(2, 2), stride=2)

  res = res.flatten()

  weights, bias = layers[3]  # this is fixed-point weights
  res = dense_FxPt(res, weights, bias, fraction_length=fraction_length)

  # res = np.float32(res) / (1 << fraction_length)
  # res = softmax(res)

  return res

In [17]:
def toFxPt16(x, fraction_length=8):
  return np.int16(x * (1 << fraction_length))

In [18]:
def layers2FxPt16(layers, fraction_length=8):
  layers_FxPt16 = []
  for layer in layers:
    l = []
    for weight in layer.get_weights():
      l.append(toFxPt16(weight))
    layers_FxPt16.append(l)
  
  return tuple(layers_FxPt16)

In [19]:
# convolution for fixed point data
def conv2d_FxPt16(input_data, filter, bias=None, stride=1, activation=True, fraction_length=8):
  '''
  Param:
  "input_data"  is an img of shape: (H, W, C)
  "filter"      is a conv kernel (H, W, C, N)
  "stride"      is the moving step size of filter.
  "activation"  Assign True for adding ReLu.
  "fracttion_len" is the length of fractional part. Max size is 16 avoiding overflow.
  Return:
  "res"         is the img after convolution
  '''

  # Use im2col first to turn input_data into matrix 
  # then apply matrix multiplication
  filter_h, filter_w, filter_c, filter_num = filter.shape
  input_mtx, (out_h, out_w) = im2col(input_data, filter_h, filter_w, stride)
  input_mtx = input_mtx.astype(dtype=np.int32)  # extend to 32 bits first
  
  # Turn filters into matrix
  filter_mtx = filter.reshape(filter_h * filter_w * filter_c, filter_num)
  filter_mtx = filter_mtx.astype(dtype=np.int32)  # extend to to 32 bits first

  # yuyu: we will replace this with our accelerated version of matrix-to-matirx
  # mutiplication

  res = np.dot(input_mtx, filter_mtx)  # this will be 32-bit mmm
  res[:] = res[:] >> fraction_length   # shift the point back to 16 bit
  res = res.astype(dtype=np.int16)     # shrink the result back to 16 bits

  if bias is not None:
    bias = bias.astype(dtype=np.int16)  # make sure it is 16-bit
    for i in range(filter_num):
      res[:, i] = res[:, i] + bias[i]

  if activation:
    res = np.maximum(res, 0)

  res = col2im(res, out_h, out_w)  # Use yuyu's version

  return res

In [20]:
# dense for fixed point data
def dense_FxPt16(input_data, weights, bias=None, fraction_length=8):
  '''
  input_data is an array with len = n
  weights    is the weights of the FC layer with shape (n, m)
  bias       is the bias of the FC layer with len = m
  fracttion_length is the length of fractional part. Max size is 16 avoiding overflow.
  '''
  input_data = input_data.astype(dtype=np.int32)
  weights = weights.astype(dtype=np.int32)
  
  _, out_n = weights.shape
  np.transpose(input_data)

  ## yuyu: we can also accelerate this part? XD
  res = np.dot(input_data, weights)
  res[:] = res[:] >> fraction_length
  res = res.astype(dtype=np.int16)


  if bias is not None:
    assert(len(bias) == out_n)
    bias = bias.astype(dtype=np.int16)
    res = res + bias
  
  return res

In [21]:
def inference_FxPt16(input_data, layers, fraction_length=4):
  '''
  input_data is an img of shape (28, 28, 1)
  layers     is layers with pretrained weights in the model
  '''
  assert(input_data.shape == (28, 28, 1))

  filter, bias = layers[0]  # this is fixed-point weights
  res = conv2d_FxPt16(input_data, filter, bias, stride=1, activation=True,
                    fraction_length=fraction_length)

  res = maxpooling2d(res, pool_size=(2, 2), stride=2)
  res = res.astype(np.int16)

  res = res.flatten()
  res = res.astype(np.int16)

  weights, bias = layers[3]  # this is fixed-point weights
  res = dense_FxPt16(res, weights, bias, fraction_length=fraction_length)

  # res = np.float32(res) / (1 << fraction_length)
  # res = softmax(res)

  return res

In [22]:
from pynq import allocate
def mtx2mem(mtx):
  '''
  mem has shape (m, n)
  output 16-bit data memory address (a contiguous memory space)
  '''
  m, n = mtx.shape
  l = -(n // -8)
  mtx = np.pad(mtx, [(0, 0), (0, l * 8 - n)], mode='constant')
  mem = allocate((m * l, 8), dtype='u2')  # This might have to change to pynq.Allocate
  for i in range(0, l):
    mem[m * i : m * (i + 1), :] = mtx[:, 8 * i : 8 * (i + 1)]
  return mem

In [23]:
def mem2mtx(mem, m, n):
  '''
  mem assumes a 16-bit integer memory layout allocated using pynq.Allocate
  m, n is the matrix shape
  '''
  l = -(n // -8)
  mtx = np.zeros((m, l * 8), dtype=np.int16)
  for i in range(0, l):
     mtx[:, 8 * i : 8 * (i + 1)] = mem[m * i : m * (i + 1), :]
  mtx = mtx[:, 0 : n]
  return mtx

In [24]:
# Testing mem2mtx and mtx2mem
m, n = 9, 9
test = np.arange(m * n).reshape(m, n)
print(test)
mem_test = mtx2mem(test)
print(mem_test)
print(mem2mtx(mem_test, m, n))

[[ 0  1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16 17]
 [18 19 20 21 22 23 24 25 26]
 [27 28 29 30 31 32 33 34 35]
 [36 37 38 39 40 41 42 43 44]
 [45 46 47 48 49 50 51 52 53]
 [54 55 56 57 58 59 60 61 62]
 [63 64 65 66 67 68 69 70 71]
 [72 73 74 75 76 77 78 79 80]]
[[ 0  1  2  3  4  5  6  7]
 [ 9 10 11 12 13 14 15 16]
 [18 19 20 21 22 23 24 25]
 [27 28 29 30 31 32 33 34]
 [36 37 38 39 40 41 42 43]
 [45 46 47 48 49 50 51 52]
 [54 55 56 57 58 59 60 61]
 [63 64 65 66 67 68 69 70]
 [72 73 74 75 76 77 78 79]
 [ 8  0  0  0  0  0  0  0]
 [17  0  0  0  0  0  0  0]
 [26  0  0  0  0  0  0  0]
 [35  0  0  0  0  0  0  0]
 [44  0  0  0  0  0  0  0]
 [53  0  0  0  0  0  0  0]
 [62  0  0  0  0  0  0  0]
 [71  0  0  0  0  0  0  0]
 [80  0  0  0  0  0  0  0]]
[[ 0  1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16 17]
 [18 19 20 21 22 23 24 25 26]
 [27 28 29 30 31 32 33 34 35]
 [36 37 38 39 40 41 42 43 44]
 [45 46 47 48 49 50 51 52 53]
 [54 55 56 57 58 59 60 61 62]
 [63 64 65 66 67 68 69 70 71]
 [

In [25]:
input_i = np.load('FxPt_input_file.npy')

In [26]:
# This part can re-built the weight tuple
y = np.load('FxPt_L0.npz')
x = np.load('FxPt_L3.npz')
LOAD_layers = []
LOAD_layers.append([y['L0_weight'], y['L0_bias']])
LOAD_layers.append([])
LOAD_layers.append([])
LOAD_layers.append([x['L3_weight'], x['L3_bias']])
LOAD_layers = tuple(LOAD_layers)

In [27]:
def HW_inference_FxPt16(input_data, layers, fraction_length=4):
  '''
  input_data is an img of shape (28, 28, 1)
  layers     is layers with pretrained weights in the model
  '''
  assert(input_data.shape == (28, 28, 1))

  filter, bias = layers[0]  # this is fixed-point weights
  res = HW_conv2d_FxPt16(input_data, filter, bias, stride=1, activation=True,
                    fraction_length=fraction_length)

  res = maxpooling2d(res, pool_size=(2, 2), stride=2)
  res = res.astype(np.int16)

  res = res.flatten()
  res = res.astype(np.int16)

  weights, bias = layers[3]  # this is fixed-point weights
  res = dense_FxPt16(res, weights, bias, fraction_length=fraction_length)

  # res = np.float32(res) / (1 << fraction_length)
  # res = softmax(res)

  return res

In [28]:
def HW_conv2d_FxPt16(input_data, filter, bias=None, stride=1, activation=True, fraction_length=8):
  '''
  Param:
  "input_data"  is an img of shape: (H, W, C)
  "filter"      is a conv kernel (H, W, C, N)
  "stride"      is the moving step size of filter.
  "activation"  Assign True for adding ReLu.
  "fracttion_len" is the length of fractional part. Max size is 16 avoiding overflow.
  Return:
  "res"         is the img after convolution
  '''

  # Use im2col first to turn input_data into matrix 
  # then apply matrix multiplication
  filter_h, filter_w, filter_c, filter_num = filter.shape
  input_mtx, (out_h, out_w) = im2col(input_data, filter_h, filter_w, stride)
  input_mtx = input_mtx.astype(dtype=np.int32)  # extend to 32 bits first
  
  # Turn filters into matrix
  filter_mtx = filter.reshape(filter_h * filter_w * filter_c, filter_num)
  filter_mtx = filter_mtx.astype(dtype=np.int32)  # extend to to 32 bits first

  # yuyu: we will replace this with our accelerated version of matrix-to-matirx
  # mutiplication

  res = mtx_mult_acc(input_mtx, filter_mtx)  # this will be 32-bit mmm
  res[:] = res[:] >> fraction_length   # shift the point back to 16 bit
  res = res.astype(dtype=np.int16)     # shrink the result back to 16 bits

  if bias is not None:
    bias = bias.astype(dtype=np.int16)  # make sure it is 16-bit
    for i in range(filter_num):
      res[:, i] = res[:, i] + bias[i]

  if activation:
    res = np.maximum(res, 0)

  res = col2im(res, out_h, out_w)  # Use yuyu's version

  return res

In [29]:
def mtx_mult_acc(mtx_a, mtx_b):
    
    m, ki = mtx_a.shape
    kf, n = mtx_b.shape
    assert(ki == kf)
    
    mem_a = mtx2mem(mtx_a.transpose())
    mem_b = mtx2mem(mtx_b)
    print(mem_a)
    print(mem_b)
    
    CDMA_MMIO.write(0x18, mem_a.device_address)  # src
    CDMA_MMIO.write(0x20, BRAM0_ADDR)     # dst
    CDMA_MMIO.write(0x28, mem_a.shape[0] * mem_a.shape[1] * 2)    # size.
    CDMA_MMIO.write(0x00, 0x04)    # Start
    idle = CDMA_MMIO.read(0x04) & 0x2
    while (idle == 0):
        print(1)
        idle = CDMA_MMIO.read(0x04) & 0x2
    
    CDMA_MMIO.write(0x18, mem_b.device_address)  # src
    CDMA_MMIO.write(0x20, BRAM1_ADDR)     # dst
    CDMA_MMIO.write(0x28, mem_b.shape[0] * mem_b.shape[1] * 2)    # size.
    CDMA_MMIO.write(0x00, 0x04)    # Start
    idle = CDMA_MMIO.read(0x04) & 0x2
    while (idle == 0):
        print(2)
        idle = CDMA_MMIO.read(0x04) & 0x2
    
    print(mem_a.shape[0] * mem_a.shape[1] * 2)
    print(mem_b.shape[0] * mem_b.shape[1] * 2)


    TPU_MMIO.write(8, m)
    TPU_MMIO.write(12, ki)
    TPU_MMIO.write(16, n)
    
    # bram baseAddr
    TPU_MMIO.write(20, 0)
    TPU_MMIO.write(24, 0)
    TPU_MMIO.write(28, 0)

    # start
    TPU_MMIO.write(0, 1)
    
    
    # busy waiting
    valid = TPU_MMIO.read(0x4)
    while(valid == 0):
        valid = TPU_MMIO.read(0x4)
    
    # reconstruct the array
    l = -(n // -8)
    mem_p = allocate((l * m, 8), dtype='u2')
    CDMA_MMIO.write(0x18, BRAM2_ADDR)     # src
    CDMA_MMIO.write(0x20, mem_p.device_address)  # dst
    CDMA_MMIO.write(0x28, mem_p.shape[0] * mem_p.shape[1] * 2)    # size.
    CDMA_MMIO.write(0x00, 0x04)    # Start
    idle = CDMA_MMIO.read(0x04) & 0x2
    while (idle == 0):
        idle = CDMA_MMIO.read(0x04) & 0x2
    res = mem2mtx(mem_p, m , n)
    print(mem_p)
    
    TPU_MMIO.write(0,0)
    
    return res

In [30]:
import matplotlib.pyplot as plt
plt.imshow(input_i[:, :, 0])

In [31]:
res_software = inference_FxPt16(input_i, LOAD_layers, fraction_length = 8)
print(np.argmax(res_software))

6


In [32]:
res = HW_inference_FxPt16(input_i, LOAD_layers, fraction_length = 8)
print(np.argmax(res))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[65195    40   190 65418    91    83    65    68]
 [65514    62    19     8   225 65447    40 65442]
 [  225    61 65407   109   201    42    48 65383]
 [65256    62 65452 65511 65491    59 65498    90]
 [   98 65529 65428   127    86 65434    39    40]
 [  156 65529 65532 65534    78    35    23    52]
 [65441    64 65437    58 65198    43    75    29]
 [   82 65509    93    79 65206 65516    15    86]
 [   79 65534   115 65439 65274    62     7    69]
 [   78    72 65216    29     0     0     0     0]
 [  144 65518 65224    16     0     0     0     0]
 [   25 65427 65226 65478     0     0     0     0]
 [65422    54 65295 65504     0     0     0     0]
 [  141   109    61   105     0     0     0     0]
 [  157 65528   119    98     0     0     0     0]
 [65294 65455    93 65533     0     0     0     0]
 [65372    54   167    22     0     0     0     0]
 [   68    73

In [33]:
print(res)
print(res_software)

[ -462  1454    99  -373  -309   653  -256   603 -1928  -930]
[-1979 -1147 -1599 -1886 -1050 -1960  2176 -2632 -1649 -2065]


In [103]:
m, k, n = 8, 8, 8
mtx_a = np.arange(m * k).reshape(m, k).astype(np.int16)
mtx_b = np.arange(k * n).reshape(k, n).astype(np.int16)

res = mtx_mult_acc(mtx_a, mtx_b)
out = allocate((m, n), dtype='u2')
CDMA_MMIO.write(0x20, out.device_address)  # dst
CDMA_MMIO.write(0x18, BRAM0_ADDR)     # src
CDMA_MMIO.write(0x28, m * n * 2)    # size.
CDMA_MMIO.write(0x00, 0x04)    # Start
idle = CDMA_MMIO.read(0x04) & 0x2

print(res)  # sadly its all zero
print(np.dot(mtx_a, mtx_b))
print(BRAM2_MMIO.read(0x0))
print(out)

[[ 0  8 16 24 32 40 48 56]
 [ 1  9 17 25 33 41 49 57]
 [ 2 10 18 26 34 42 50 58]
 [ 3 11 19 27 35 43 51 59]
 [ 4 12 20 28 36 44 52 60]
 [ 5 13 21 29 37 45 53 61]
 [ 6 14 22 30 38 46 54 62]
 [ 7 15 23 31 39 47 55 63]]
[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]]
128
128
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[ 1120  1148  1176  1204  1232  1260  1288  1316]
 [ 2912  3004  3096  3188  3280  3372  3464  3556]
 [ 4704  4860  5016  5172  5328  5484  5640  5796]
 [ 6496  6716  6936  7156  7376  7596  7816  8036]
 [ 8288  8572  8856  9140  9424  9708  9992 1027